In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [ ]:
model = AutoModelForCausalLM.from_pretrained("D:\CoachBot\SFTuned")
tokenizer=AutoTokenizer.from_pretrained("D:\CoachBot\SFTuned", use_fast=False)

In [ ]:
model = model.to("cuda")
model.eval()

In [ ]:
'''import torch
# 2. 单轮对话函数
def chat_qwen3(messages, 
               enable_thinking: bool = True, 
               max_new_tokens: int = 512):
    """
    messages: List[Dict[str, str]]，格式如 [{"role":"user","content": "..."}]
    enable_thinking: 是否开启 <think> 模式
    """
    # 2.1 应用聊天模板，拼接 prompt + 回复提示
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking
    )
    # 2.2 编码并送入模型
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    # 2.3 生成
    generation = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
        top_k=20
    )
    # 2.4 切掉输入部分，只保留模型新生成的 token
    output_ids = generation[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

# 3. 测试单轮
messages = [{"role": "user", "content": "我好难受。"}]
response = chat_qwen3(messages)
print("Bot:", response)'''

In [5]:
import torch
# 4. 多轮示例（可选）
class QwenChatbot:
    def __init__(self, 
                 model_name="D:\CoachBot\SFTuned",
                 system_prompt="你是一位心理咨询师。"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")
        self.history = [{"role": "system", "content": system_prompt}]
    
    def generate(self, user_input, enable_thinking=True):
        self.history.append({"role": "user", "content": user_input})
        text = self.tokenizer.apply_chat_template(
            self.history,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=enable_thinking
        )
        inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        gen = self.model.generate(**inputs, max_new_tokens=512)
        out_ids = gen[0][len(inputs.input_ids[0]):]
        reply = self.tokenizer.decode(out_ids, skip_special_tokens=True)
        self.history.append({"role": "assistant", "content": reply})
        return reply

In [ ]:
# 使用多轮 Chatbot
bot = QwenChatbot()
print("Bot:", bot.generate("我好难受", enable_thinking=True))

In [6]:
if __name__ == "__main__":
    bot = QwenChatbot("D:/CoachBot/SFTuned")
    print("=== Qwen3 Chat ===")
    while True:
        q = input("You: ")
        if q.strip().lower() in ("exit", "quit"):
            print("Bye")
            break
        # 你可以根据需要切换 thinking 模式
        ans = bot.generate(q, enable_thinking=False)
        print("Bot:", ans)

=== Qwen3 Chat ===
Bot: 你好，我听到你说“好难受”。这说明你可能正在经历一些痛苦或不适。我想，我们可以先来个简单的对话，了解更多的背景信息和感受，然后根据这些信息来设计一个合适的心理咨询方案。咨询过程是安全的，你可以随时退出，或者向其他咨询者求助。最重要的是，我希望你能在这个过程中找到一个能够理解和支持你的的人。谢谢你的信任，也希望你能从中学到一些东西。下一次，我们准备好了吗？
Bot: 你好，很高兴见到你。心理咨询是一种专业领域，旨在帮助个体或群体解决内心的问题、提升自我意识、促进成长与发展。在这里，你可以得到一个安全、保密、专业的服务。我们还需要一个合格的咨询师，这由你的选择决定。因此，在这里，我们可以暂时放下之前的对话，专注于当前的互动。下次，当你再次提到“好难受”时，我将再次向你承诺，确保服务是安全、有效的。记住，心理咨询并不是解决问题的万能药，但它能为许多其他方面带来积极的变化。谢谢你，期待着与你的对话。下一次见，祝你一切顺利。
Bot: 谢谢你分享了你的感受，这让我能够更好地了解和回应你的需求。之前的对话只是用来建立一个安全的咨询环境，之后的对话将由你主导，由你提出问题和要求，由我协助执行。因此，现在可以随时结束对话，或者向其他咨询者求助。记住，心理咨询是一个持续的过程，需要时间和耐心。期待着与你的再次相遇，祝你一切顺利。下次见，你好。
Bye
